# Face Recognition Application with Webcam

This notebook implements a face recognition system using eigenfaces with PCA or Bayesian models. It allows users to:
- Use a webcam for real-time face recognition (requires browser camera permission)
- Upload images as a fallback if webcam access is unavailable
- Train PCA or Bayesian models
- Manage a face database (add subjects, update labels)

**Note**: Webcam access requires browser permission. If the webcam stream doesn't work, use the image upload option. Ensure you run this in a MyBinder environment with the required dependencies.

In [ ]:
# Install required packages
!pip install opencv-python-headless numpy mediapipe pillow ipywidgets ipywebrtc

# Enable ipywidgets
!jupyter nbextension enable --py widgetsnbextension

import cv2
import numpy as np
import mediapipe as mp
import random
from PIL import Image
from collections import deque
import ipywidgets as widgets
from IPython.display import display, Image as IPImage, clear_output
import io
import base64
try:
    from ipywebrtc import CameraStream, ImageRecorder
except ImportError:
    CameraStream = None
    ImageRecorder = None

# Eigenfaces utility functions
def compute_eigenfaces(images, image_size):
    images_flat = images.reshape(images.shape[0], -1)
    mean = np.mean(images_flat, axis=0)
    images_centered = images_flat - mean
    cov = np.cov(images_centered.T)
    eigenvalues, eigenvectors = np.linalg.eigh(cov)
    idx = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    return eigenvectors.T, mean

def project_images(images, mean, eigenfaces, num_components, image_size):
    images_flat = images.reshape(images.shape[0], -1)
    images_centered = images_flat - mean
    return np.dot(images_centered, eigenfaces[:num_components].T)

def calculate_class_statistics(trainE, train_labels):
    class_means = {}
    class_covariances = {}
    for label in set(train_labels):
        indices = train_labels == label
        class_data = trainE[indices]
        class_means[label] = np.mean(class_data, axis=0)
        class_covariances[label] = np.cov(class_data.T)
    return class_means, class_covariances

def predict_single_image_bayes(image, mean, eigenfaces, class_means, class_covariances, image_size, num_components):
    image_flat = image.flatten() - mean
    projection = np.dot(image_flat, eigenfaces[:num_components].T)
    probs = {}
    for label in class_means:
        mean_diff = projection - class_means[label]
        cov_inv = np.linalg.pinv(class_covariances[label])
        exponent = -0.5 * np.dot(mean_diff.T, np.dot(cov_inv, mean_diff))
        probs[label] = exponent
    return max(probs, key=probs.get), probs

def predict_single_image(image, mean, eigenfaces, trainE, train_labels, image_size, num_components):
    image_flat = image.flatten() - mean
    projection = np.dot(image_flat, eigenfaces[:num_components].T)
    distances = np.linalg.norm(trainE - projection, axis=1)
    min_idx = np.argmin(distances)
    return train_labels[min_idx]

# FaceRecognitionApp class
class FaceRecognitionApp:
    def __init__(self):
        self.image_size = (112, 92)
        self.num_components = 4
        self.prob_threshold = 0.5
        self.model_type = 'Bayesian'
        self.running = False
        self.prediction_buffer = deque(maxlen=5)
        self.train_images = np.array([])
        self.train_labels = np.array([])
        self.test_images = np.array([])
        self.test_labels = np.array([])
        self.eigenfaces = None
        self.mean = None
        self.trainE = None
        self.class_means = {}
        self.class_covariances = {}

        self.mp_face_detection = mp.solutions.face_detection
        self.face_detector = self.mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.6)

        # Widgets
        self.model_dropdown = widgets.Dropdown(options=['Bayesian', 'PCA'], value='Bayesian', description='Model:')
        self.components_input = widgets.IntText(value=4, description='Components:', layout={'width': '200px'})
        self.apply_button = widgets.Button(description='Apply Settings')
        self.start_webcam_button = widgets.Button(description='Start Webcam')
        self.stop_webcam_button = widgets.Button(description='Stop Webcam', disabled=True)
        self.upload_button = widgets.FileUpload(accept='.jpg,.png', multiple=True, description='Upload Images')
        self.process_upload_button = widgets.Button(description='Process Uploaded Images')
        self.add_subject_button = widgets.Button(description='Add New Subject')
        self.update_label_button = widgets.Button(description='Update Label')
        self.name_input = widgets.Text(description='Name:', layout={'width': '300px'})
        self.current_label_dropdown = widgets.Dropdown(options=[], description='Current Label:')
        self.new_label_input = widgets.Text(description='New Label:', layout={'width': '300px'})
        self.output = widgets.Output()

        # Webcam setup
        self.camera = None
        self.recorder = None
        if CameraStream is not None:
            try:
                self.camera = CameraStream(constraints={'video': True, 'audio': False})
                self.recorder = ImageRecorder(stream=self.camera)
            except Exception as e:
                with self.output:
                    print(f'Error initializing webcam: {e}')

        # Callbacks
        self.apply_button.on_click(self.update_settings)
        self.start_webcam_button.on_click(self.start_webcam)
        self.stop_webcam_button.on_click(self.stop_webcam)
        self.process_upload_button.on_click(self.process_uploaded_images)
        self.add_subject_button.on_click(self.add_new_subject)
        self.update_label_button.on_click(self.update_label)
        self.model_dropdown.observe(self.update_model, names='value')
        if self.recorder:
            self.recorder.observe(self.process_webcam_frame, names='image')

    def load_images(self, images, labels):
        self.train_images = np.array(images)
        self.train_labels = np.array(labels)
        if len(self.train_images) > 0:
            self.retrain_model()
            self.current_label_dropdown.options = list(self.class_means.keys())

    def retrain_model(self):
        if self.train_images.size == 0:
            with self.output:
                print('No training data available')
            return

        if self.num_components > len(self.train_images):
            with self.output:
                print('Number of components exceeds number of training images')
            return

        self.eigenfaces, self.mean = compute_eigenfaces(self.train_images, self.image_size)
        if self.num_components > self.eigenfaces.shape[0]:
            with self.output:
                print('Number of components exceeds number of eigenfaces')
            return

        self.trainE = project_images(self.train_images, self.mean, self.eigenfaces, self.num_components, self.image_size)
        self.class_means = {}
        self.class_covariances = {}
        if self.model_type == 'Bayesian':
            self.class_means, self.class_covariances = calculate_class_statistics(self.trainE, self.train_labels)

    def update_model(self, change):
        self.model_type = change['new']
        self.retrain_model()
        with self.output:
            print(f'Switched to {self.model_type} model')

    def update_settings(self, b):
        try:
            new_components = self.components_input.value
            if new_components < 1:
                raise ValueError('Number of components must be positive')
            if self.train_images.size > 0 and new_components > len(self.train_images):
                raise ValueError('Number of components cannot exceed number of training images')
            if self.eigenfaces is not None and new_components > self.eigenfaces.shape[0]:
                raise ValueError('Number of components cannot exceed number of eigenfaces')
            self.num_components = new_components
            self.retrain_model()
            with self.output:
                print(f'Updated to {self.num_components} components')
        except ValueError as e:
            with self.output:
                print(f'Error: {e}')

    def process_face(self, face_resized):
        if self.eigenfaces is None or self.mean is None:
            return 'Desconocido', None

        if self.model_type == 'Bayesian':
            label, probs = predict_single_image_bayes(
                face_resized, self.mean, self.eigenfaces,
                self.class_means, self.class_covariances,
                self.image_size, self.num_components
            )
            probs_exp = {k: np.exp(v) for k, v in probs.items()}
            total = sum(probs_exp.values())
            probs_percent = {k: (v / total) * 100 for k, v in probs_exp.items()}
            label = max(probs_percent, key=probs_percent.get)
            confidence = probs_percent[label] / 100
            predicted_label = label if confidence >= self.prob_threshold else 'Desconocido'
        else:
            label = predict_single_image(
                face_resized, self.mean, self.eigenfaces,
                self.trainE, self.train_labels,
                self.image_size, self.num_components
            )
            predicted_label = label if label else 'Desconocido'
            confidence = None

        return predicted_label, confidence

    def smooth_prediction(self, current_label, current_confidence):
        if current_label != 'Desconocido':
            self.prediction_buffer.append((current_label, current_confidence))

        if len(self.prediction_buffer) == 0:
            return current_label, current_confidence

        labels = [label for label, _ in self.prediction_buffer]
        confidences = [conf for _, conf in self.prediction_buffer if conf is not None]

        most_common_label = max(set(labels), key=labels.count)
        avg_confidence = sum(confidences) / len(confidences) if confidences else None

        return most_common_label, avg_confidence

    def process_webcam_frame(self, change):
        if not self.running:
            return

        with self.output:
            clear_output(wait=True)
            try:
                # Decode image from recorder
                img_data = change['new']
                img_array = np.frombuffer(img_data, np.uint8)
                frame = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
                if frame is None:
                    print('Error decoding webcam frame')
                    return

                rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = self.face_detector.process(rgb)
                last_result = []

                if results.detections:
                    for d in results.detections:
                        bbox = d.location_data.relative_bounding_box
                        h, w, _ = frame.shape
                        x, y = int(bbox.xmin * w), int(bbox.ymin * h)
                        w_box, h_box = int(bbox.width * w), int(bbox.height * h)
                        x, y = max(0, x), max(0, y)
                        face = cv2.cvtColor(frame[y:y+h_box, x:x+w_box], cv2.COLOR_BGR2GRAY)
                        if face.size == 0:
                            continue
                        face_resized = cv2.resize(face, (self.image_size[1], self.image_size[0]))

                        predicted_label, confidence = self.process_face(face_resized)
                        predicted_label, confidence = self.smooth_prediction(predicted_label, confidence)

                        last_result.append(((x, y, w_box, h_box), predicted_label, confidence))
                else:
                    self.prediction_buffer.clear()

                for (x, y, w_box, h_box), name, conf in last_result:
                    cv2.rectangle(frame, (x, y), (x + w_box, y + h_box), (0, 255, 0), 2)
                    cv2.putText(frame, f'{name}', (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
                    if conf is not None:
                        cv2.putText(frame, f'prob={round(conf, 2)}', (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

                # Display frame
                _, buffer = cv2.imencode('.jpg', frame)
                display(IPImage(data=buffer.tobytes()))

            except Exception as e:
                print(f'Error processing webcam frame: {e}')

    def start_webcam(self, b):
        if self.running:
            with self.output:
                print('Webcam is already running')
            return

        if self.camera is None or self.recorder is None:
            with self.output:
                print('Webcam not available. Please use image upload instead.')
            return

        try:
            self.running = True
            self.start_webcam_button.disabled = True
            self.stop_webcam_button.disabled = False
            self.recorder.recording = True
            with self.output:
                print('Webcam started. Allow camera access if prompted.')
        except Exception as e:
            with self.output:
                print(f'Error starting webcam: {e}')
                self.running = False
                self.start_webcam_button.disabled = False
                self.stop_webcam_button.disabled = True

    def stop_webcam(self, b):
        if not self.running:
            return

        self.running = False
        if self.recorder:
            self.recorder.recording = False
        self.start_webcam_button.disabled = False
        self.stop_webcam_button.disabled = True
        with self.output:
            clear_output()
            print('Webcam stopped')

    def process_uploaded_images(self, b):
        with self.output:
            clear_output()
            if not self.upload_button.value:
                print('No images uploaded')
                return

            for name, file_info in self.upload_button.value.items():
                img_data = file_info['content']
                img_array = np.frombuffer(img_data, np.uint8)
                frame = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
                if frame is None:
                    print(f'Error loading image {name}')
                    continue

                rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = self.face_detector.process(rgb)
                if results.detections:
                    for d in results.detections:
                        bbox = d.location_data.relative_bounding_box
                        h, w, _ = frame.shape
                        x, y = int(bbox.xmin * w), int(bbox.ymin * h)
                        w_box, h_box = int(bbox.width * w), int(bbox.height * h)
                        x, y = max(0, x), max(0, y)
                        face = cv2.cvtColor(frame[y:y+h_box, x:x+w_box], cv2.COLOR_BGR2GRAY)
                        if face.size == 0:
                            continue
                        face_resized = cv2.resize(face, (self.image_size[1], self.image_size[0]))

                        predicted_label, confidence = self.process_face(face_resized)
                        predicted_label, confidence = self.smooth_prediction(predicted_label, confidence)

                        cv2.rectangle(frame, (x, y), (x + w_box, y + h_box), (0, 255, 0), 2)
                        cv2.putText(frame, f'{predicted_label}', (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
                        if confidence is not None:
                            cv2.putText(frame, f'prob={round(confidence, 2)}', (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

                    # Display image
                    _, buffer = cv2.imencode('.jpg', frame)
                    display(IPImage(data=buffer.tobytes()))
                else:
                    print(f'No faces detected in {name}')

    def add_new_subject(self, b):
        with self.output:
            clear_output()
            label = self.name_input.value
            if not label:
                print('Please enter a name')
                return
            if not self.upload_button.value:
                print('Please upload at least 5 images')
                return

            new_images = []
            new_labels = []

            for name, file_info in self.upload_button.value.items():
                img_data = file_info['content']
                img_array = np.frombuffer(img_data, np.uint8)
                img = cv2.imdecode(img_array, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    continue

                rgb_img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                results = self.face_detector.process(rgb_img)
                if not results.detections:
                    print(f'No face detected in {name}')
                    continue

                bbox = results.detections[0].location_data.relative_bounding_box
                h, w = img.shape
                x = int(bbox.xmin * w)
                y = int(bbox.ymin * h)
                width = int(bbox.width * w)
                height = int(bbox.height * h)
                x, y = max(0, x), max(0, y)
                face_roi = img[y:y+height, x:x+width]
                if face_roi.size == 0:
                    continue
                face_resized = cv2.resize(face_roi, (self.image_size[1], self.image_size[0]))
                new_images.append(face_resized)
                new_labels.append(label)

            if len(new_images) >= 5:
                new_images_array = np.array(new_images)
                if self.train_images.size == 0:
                    self.train_images = new_images_array
                    self.train_labels = np.array(new_labels)
                else:
                    self.train_images = np.concatenate((self.train_images, new_images_array), axis=0)
                    self.train_labels = np.concatenate((self.train_labels, np.array(new_labels)), axis=0)
                self.retrain_model()
                self.current_label_dropdown.options = list(self.class_means.keys())
                print(f'Subject {label} added successfully')
            else:
                print('Not enough valid face images detected (need at least 5)')

    def update_label(self, b):
        with self.output:
            clear_output()
            old_label = self.current_label_dropdown.value
            new_label = self.new_label_input.value
            if not old_label or not new_label:
                print('Please fill in both fields')
                return

            if new_label in self.class_means:
                print(f'Label \'{new_label}\' already exists.')
                return

            self.train_labels = np.where(self.train_labels == old_label, new_label, self.train_labels)
            self.retrain_model()
            self.current_label_dropdown.options = list(self.class_means.keys())
            print(f'Label updated from {old_label} to {new_label}')

    def display_ui(self):
        webcam_controls = widgets.HBox([self.start_webcam_button, self.stop_webcam_button]) if self.camera else widgets.Label('Webcam not available. Use image upload.')
        display(widgets.VBox([
            widgets.HBox([self.model_dropdown, self.components_input, self.apply_button]),
            webcam_controls,
            self.upload_button,
            self.process_upload_button,
            widgets.HBox([self.name_input, self.add_subject_button]),
            widgets.HBox([self.current_label_dropdown, self.new_label_input, self.update_label_button]),
            self.output
        ]))

# Initialize and display the app
app = FaceRecognitionApp()
app.display_ui()

## Instructions

1. **Model Selection**: Choose between 'Bayesian' and 'PCA' models using the dropdown.
2. **Components**: Set the number of eigenface components and click 'Apply Settings'.
3. **Webcam**: Click 'Start Webcam' to begin real-time face recognition. Allow camera access when prompted. Click 'Stop Webcam' to end.
4. **Image Upload**: If webcam access fails, upload images (.jpg or .png) and click 'Process Uploaded Images'.
5. **Add Subject**: Enter a name, upload at least 5 images, and click 'Add New Subject' to add to the database.
6. **Update Label**: Select a current label, enter a new label, and click 'Update Label' to rename a subject.

**Notes**:
- The database is stored in memory and resets when the notebook restarts.
- Webcam access requires browser permission and may not work in all MyBinder sessions due to resource constraints.
- For persistent storage, consider integrating a cloud storage service (not included in this version).